In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import wordcloud
import re

from bs4 import BeautifulSoup 
from collections import Counter
from wordcloud import WordCloud, STOPWORDS

In [2]:
### First website that I am scraping. This website has all beatles songs, 
# who wrote them, and who sang them

def get_rid(words, clean = None):
    # Need this so we do not reset clean at each recursion
    if clean is None:
        clean = [] 

    for i in range(len(words)):
        if "total" in words[i].lower():
            j = i
            continue
        if "singer" in words[i].lower():
            k = i
            keep1 = words[:j]
            keep2 = words[i+1:]
            clean.extend(keep1)

            return get_rid(keep2, clean = clean)
    return clean 

## Get the data
url = "https://www.myrsten.nu/worldnet/beatlesongs.htm"
res = requests.get(url)
htmlData = res.content
#print(htmlData)

## Parse the Data
soup = BeautifulSoup(htmlData, "html.parser")

# Find all <td> elements
td_elements = soup.find_all("td")

# We want to store song, composer, and singer. 
songs = []

# Skip headers, then process title, composer, and singer
td_texts = [td.get_text(strip=True) for td in td_elements[4:]]  # Skip headers
#print(td_texts)

# One part where the text is a different langauge, change that
index1 = td_texts.index("Låt")
index2 = td_texts.index("Huvudkompositör")
index3 = td_texts.index("Sångare")

td_texts[index1] = "Song"
td_texts[index2] = "Main composer"
td_texts[index3] = "Singer"

# There is some text that we want to remove before saving song_title, composer, and singer
clean = get_rid(td_texts)

for i in range(0, len(clean), 3):
    if i + 2 < len(clean):
        song_title = clean[i]
        composer = clean[i + 1]
        singer = clean[i + 2]

        # Some cleaning
        song_title = song_title.replace("\n", " ")
        composer = composer.replace("\n", " ")
        singer = singer.replace("\n", " ")
        song_title = re.sub(r'\"', ' ', song_title, flags = re.IGNORECASE)
        composer = re.sub(r'\"', ' ', composer, flags = re.IGNORECASE)
        singer = re.sub(r'\"', ' ', singer, flags = re.IGNORECASE)
        song_title = re.sub(r',', ' ', song_title, flags = re.IGNORECASE)
        composer = re.sub(r',', ' ', composer, flags = re.IGNORECASE)
        singer = re.sub(r',', ' ', singer, flags = re.IGNORECASE)
        
        songs.append([song_title, composer, singer])

# Write to csv file
filename = "song_composer_singer.csv"

# Open file
my_file = open(filename, "w")

# Write first rows, which are column names
write_this = "Song, Composer, Singer \n"
my_file.write(write_this)
my_file.close()

my_file = open(filename, "a")

for group in songs:
    song = group[0] 
    composer = group[1]
    singer = group[2]

    write_this = str(song) + "," + str(composer) + "," + str(singer) + "\n"
    my_file.write(write_this)

my_file.close()